# 연관규칙 평가척도

1. 지지도(support): 아이템 집합이 전체 트랜잭션 데이터에서 발생한 비율
    - S(A->B) = N(A, B)(트랜잭션 데이터에서 A와 B의 동시 출현 횟수) / n(트랜잭션 데이터 크기)
2. 신뢰도(confidence): 부모 아이템 집합이 등장한 트랜잭션 데이터에서 자식 아이템 집합이 발생한 비율
    - C(A->B) = N(A, B)(트랜잭션 데이터에서 A와 B의 동시 출현 횟수) / N(A)(트랜잭션 데이터에서 A의 출현 횟수)


- 지지도와 신뢰도가 높은 연관규칙을 좋은 규칙이라고 함
- 지지도에 대한 Apriori 원리
    - S(A->B)가 최소 지지도 이상이면 이 규칙을 빈발하다고 함
    - 아이템 집합의 지지도가 최소 지지도 이상이면 이 집합을 빈발하다고 함
    - 어떤 아이템 집합이 빈발하면, 이 아이템의 부분 집합도 빈발함
    - 후보 규칙 생성
        - Apriori 원리를 사용하여 모든 최대 빈발 아이템 집합을 찾은 후, 후보 규칙을 모두 생성함
        - 최대 빈발 아이템 집합: 최소 지지도 이상이면서, 이 집합의 모든 모집합이 빈발하지 않는 집합
- 신뢰도에 대한 Apriori 원리

# 1. 연관규칙 탐색

In [ ]:
import pandas as pd

df = pd.read_csv('Instacart Market Basket Analysis.csv')
df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [ ]:
df['product_id'].value_counts(normalize = True).head(100)

24852.0    0.013416
13176.0    0.011296
21137.0    0.007812
21903.0    0.006999
47626.0    0.005818
             ...   
44142.0    0.001024
40604.0    0.001020
4799.0     0.001015
46906.0    0.001011
8193.0     0.001007
Name: product_id, Length: 100, dtype: float64

In [ ]:
product_list_per_order = df.groupby('order_id')['product_id'].apply(list)
product_list_per_order

order_id
1         [49302.0, 11109.0, 10246.0, 49683.0, 43633.0, ...
36        [39612.0, 19660.0, 49235.0, 43086.0, 46620.0, ...
38        [11913.0, 18159.0, 4461.0, 21616.0, 23622.0, 3...
60                                                    [nan]
96        [20574.0, 30391.0, 40706.0, 25610.0, 27966.0, ...
                                ...                        
603287    [27845.0, 44008.0, 40001.0, 14715.0, 19895.0, ...
603293    [35958.0, 7559.0, 35951.0, 10849.0, 48222.0, 2...
603323    [43403.0, 214.0, 15424.0, 20540.0, 22629.0, 33...
603326    [18434.0, 10182.0, 27284.0, 43749.0, 4664.0, 3...
603384                                   [41844.0, 19006.0]
Name: product_id, Length: 23386, dtype: object

In [ ]:
# 구매 기록 데이터 -> one hot encoding
## from mlxtend.preprocessing import TransactionEncoder
## 연관규칙 탐사에 적절하게 거래 데이터 구조를 바꾸기 위한 라이브러리

from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()     # 인스턴스 생성

## fit(data).transform(data): data를 각 아이템의 출현 여부를 갖는 ndarray 형식으로 변환
one_hot_df = encoder.fit(product_list_per_order).transform(product_list_per_order)     # ndarray 형식
one_hot_df = pd.DataFrame(one_hot_df, columns = encoder.columns_)
one_hot_df.head()

,NaN,1.0,3.0,4.0,8.0,9.0,10.0,16.0,21.0,23.0,...,49667.0,49668.0,49670.0,49675.0,49676.0,49678.0,49680.0,49681.0,49683.0,49686.0
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
from mlxtend.frequent_patterns import *

## apriori(df, min_support): df: one hot encoding 형태의 데이터 프레임, min_support: 최소 지지도
frequent_item_df = apriori(one_hot_df, min_support = 0.003)     # 0.3% 이상 구매한 상품만 대상으로 함

In [ ]:
## association_rules(frequent_dataset, metric, min_threshold): frequent_dataset에서 찾은 연관 규칙을 데이터 프레임 형태로 반환
### metric: 연관규칙을 필터링하기 위한 유용성 척도(default = confidence), min_threshold: 지정한 metric의 최소 기준치
result = association_rules(frequent_item_df, metric = 'confidence', min_threshold = 0.1)

In [ ]:
result[['antecedents', 'consequents', 'support', 'confidence']].sort_values(by = 'confidence', ascending = False).to_csv('연관규칙탐색결과.csv', index = False)

In [ ]:
result.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(231),(12121),0.010177,0.141794,0.003164,0.310924,2.192786,0.001721,1.245445
1,(1131),(7463),0.009536,0.014496,0.003079,0.322870,22.273265,0.002941,1.455413
2,(7463),(1131),0.014496,0.009536,0.003079,0.212389,22.273265,0.002941,1.257556
3,(2256),(10739),0.028008,0.073976,0.004191,0.149618,2.022528,0.002119,1.088951
4,(2256),(12121),0.028008,0.141794,0.008167,0.291603,2.056523,0.004196,1.211476


# 2. 빈발 시퀀스 탐색(시퀀스 데이터에 대한 연관규칙 탐색)

- 시퀀스 데이터: 각 요소가 (순서, 값) 형태로 구성된 데이터. 분석 시에 반드시 순서를 고려해야 함
- 예: 로그 데이터(고객 구매 기록, 고객 여정, 웹 서핑 기록)

### 순서를 고려한 연관규칙 탐사

- 시퀀스 데이터에 대한 연관규칙 탐사에 대해서는 A -> B와 B -> A가 다른 지지도를 갖기 때문에, 같은 항목 집합으로부터 규칙을 생성할 수 없음
- 신뢰도에 대한 apriori원리는 성립함
- 따라서 개별요소(이벤트)에 다른 요소를 추가하는 방식으로 규칙을 아래와 같이 직접 찾아나가야 함
- 유니크한 요소 목록 추출 -> 빈발하는 단일 이벤트 추출 -> 이벤트 추가 및 지지도 계산 -> 최대 빈발 아이템 집합 탐색 -> 규칙 생성

### 동적 프로그래밍

- 원 문제를 작은 문제로 분할한 다음 점화식으로 만들어 재귀적인 형태로 원 문제를 해결하는 방식


In [ ]:
import numpy as np

df = pd.read_csv('페이지내_사용자_이동.csv', encoding = 'cp949')
df.head()

,고객ID,방문 페이지,순서
0,0,페이지C,1
1,0,페이지E,2
2,0,페이지B,3
3,0,페이지F,4
4,0,페이지C,5


In [ ]:
# 주문별 카트에 추가한 순서를 고려하기 위해 정렬 필요

df.sort_values(by = ['고객ID', '순서'], inplace = True)

In [ ]:
page_set = df['방문 페이지'].unique()
page_set

array(['페이지C', '페이지E', '페이지B', '페이지F', '페이지D', '페이지J', '메인', '페이지G',
       '페이지A', '페이지I', '페이지H'], dtype=object)

In [ ]:
page_sequence_per_order = df.groupby('고객ID')['방문 페이지'].apply(np.array)
page_sequence_per_order

고객ID
0     [페이지C, 페이지E, 페이지B, 페이지F, 페이지C, 페이지D, 페이지J, 메인,...
1     [페이지B, 메인, 메인, 페이지A, 페이지F, 페이지C, 페이지I, 페이지E, 페...
2     [페이지F, 페이지J, 페이지D, 페이지G, 페이지C, 메인, 페이지I, 페이지J,...
3     [페이지I, 페이지I, 페이지J, 페이지J, 페이지C, 페이지A, 페이지H, 페이지...
4     [페이지J, 페이지H, 페이지G, 페이지G, 페이지E, 페이지C, 페이지A, 페이지...
                            ...                        
95    [페이지D, 페이지H, 페이지I, 메인, 페이지B, 페이지E, 페이지F, 페이지G,...
96    [페이지E, 페이지I, 페이지E, 페이지G, 페이지A, 페이지E, 페이지D, 페이지...
97    [페이지B, 페이지B, 메인, 메인, 페이지B, 페이지F, 페이지J, 페이지D, 페...
98                               [메인, 페이지D, 페이지F, 페이지G]
99    [페이지E, 페이지H, 페이지B, 페이지D, 페이지B, 페이지A, 페이지I, 페이지...
Name: 방문 페이지, Length: 100, dtype: object

In [ ]:
from itertools import product

def contain_pattern(record, pattern, L):
  output = False
  if set(record) & set(pattern) != set(pattern):     # pattern에 포함된 모든 아이템 집합이 record에 포함된 아이템 집합에 속하지 않으면
    return False
  else:
    # 패턴에 속한 개별 아이템에 대한 위치를 미리 구하기
    pattern_index_list = [np.where(record == item)[0] for item in pattern]

    ## 가능한 모든 조합에서 위치 간 거리가 L이하면 True를 반환
    # record = [A, B, C, A, C, C], pattern = [A, B], L = 1
    # A의 위치: [0, 3], B의 위치: [1]
    # 가능한 모든 조합: [0, 1], [3, 1]
    # 가능한 모든 조합의 거리 차이: [1 - 0, 1 - 3] 중에 0번째 요소는 만족하므로 True

    for pattern_index in product(*pattern_index_list):
      distance = np.array(pattern_index)[1:] - np.array(pattern_index)[:-1]
      if sum((distance <= L) & (distance > 0)) == (len(pattern_index) - 1):
        output = True
        break

    return output

In [ ]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support = 0.01, L = 1):
  queue = []
  maximum_frequent_sequence_item = []

  # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템들만 quene에 추가시킴
  for item in item_set:
    occurence = sequence_data.apply(contain_pattern, pattern = [item], L = L).sum()
    if occurence / len(sequence_data) >= min_support:
      queue.append([item])

  while queue:
    current_pattern = queue.pop()     # 맨 마지막 값 빼기
    check_maximum_frequent = True     # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
    for item in item_set:
      occurence = sequence_data.apply(contain_pattern, pattern = current_pattern + [item], L = L).sum()
      if occurence / len(sequence_data) >= min_support:     # min_support를 넘는 패턴을 queue에 추가
        check_maximum_frequent = False
        queue.append(current_pattern + [item])

      if check_maximum_frequent and len(current_pattern) > 1:
        maximum_frequent_sequence_item.append(current_pattern)

  return maximum_frequent_sequence_item

In [ ]:
def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence = 0.5, L = 1):
  # 결과 초기화
  result = {'부모': [], '자식': [], '지지도': [], '신뢰도': []}

  for sequence_item in maximum_frequent_sequence_item:
    # A -> B에서 A, B를 모두 포함하는 가짓 수 co_occurence 계산

    